In [0]:
import librosa
import librosa.display as libdisplay
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# **import Library Penting **

In [0]:
max_pad_len = 388
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset_path = '/content/drive/My Drive/SOUND/set_a/'
metadata = pd.read_csv('/content/drive/My Drive/SOUND/DATASET_A_NEW.csv')

features = []

In [7]:
len(mfccs)

NameError: ignored

In [9]:
max_pad_len - mfccs.shape[1]

NameError: ignored

In [10]:
import glob, os
os.chdir("/content/drive/My Drive/SOUND/set_a/")


MAX_LEN_MFCC = []

for file in glob.glob("*.wav"):
    #print(file)
    #extract_features(file)

    audio,sample_rate = librosa.load(file, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #print(mfccs.shape[1])
    MAX_LEN_MFCC.append(mfccs.shape[1])

print(max(MAX_LEN_MFCC)   )

# dari data dapet 388 len nya 

388


In [0]:
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(dataset_path),str(row['fname']))
    
    class_label = row['fname']
    data = extract_features(file_name)
    
    features.append([data,class_label])

In [12]:
DATAX = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(DATAX), ' files')

Finished feature extraction from  124  files


In [0]:
str(DATAX['class_label'][0]).split('__')[0]

LABEL = []

for i in range(len(DATAX)):
  str(DATAX['class_label'][i]).split('__')[0]
  LABEL.append(str(DATAX['class_label'][i]).split('__')[0])


In [0]:
DATAX['LABEL'] = LABEL

NEW_DATAX = DATAX.drop(columns=['class_label'])

NEW_DATAX.to_csv(r'NEW_DATAX.csv')

In [15]:
NEW_DATAX

,feature,LABEL
0,"[[-629.3815027521428, -614.3528398171517, -616...",artifact
1,"[[-990.9772750518831, -990.1641333446495, -989...",artifact
2,"[[-632.0832226192613, -622.4774297438613, -623...",artifact
3,"[[-322.1109746901032, -295.3795520831543, -277...",artifact
4,"[[-294.6576987096496, -292.31416581691946, -28...",artifact
...,...,...
119,"[[-522.8667459421804, -491.1161367555086, -468...",normal
120,"[[-497.0974694148711, -494.25662561002684, -49...",normal
121,"[[-510.270796132117, -517.8475036795197, -516....",normal
122,"[[-502.076714187306, -493.8695590658039, -495....",normal


In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, utils
import matplotlib.pyplot as plt
from sklearn import metrics 

In [0]:
from sklearn.preprocessing import LabelEncoder

X = np.array(NEW_DATAX.feature.tolist())
y = np.array(NEW_DATAX.LABEL.tolist())

In [0]:
le = LabelEncoder()
yy = tf.keras.utils.to_categorical(le.fit_transform(y))

In [0]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [20]:
#Untuk menentukan column nanti di model

print('NEW_DATAX shape >> {}'.format(NEW_DATAX.shape))
print('len X_train >> {}'.format(len(x_train)))
print('len X_test >> {}'.format(len(x_test)))

# untuk dapetin num_rows, num_clumns d
print('X_train shape >> {}'.format(x_train.shape))

# len(mfccs) --> nanti kita gunakan untuk num_rows
# max(MAX_LEN_MFCC) --> dibunakan untuk num_columns & max_pad_len pada fungsi extract_features
print('Num_Rows >> {}'.format(len(mfccs)))
print('Num_Columns >> {}'.format(max(MAX_LEN_MFCC)))

NEW_DATAX shape >> (124, 2)
len X_train >> 99
len X_test >> 25
X_train shape >> (99, 40, 388)
Num_Rows >> 40
Num_Columns >> 388


# **Mempersiapkan nilai untuk Train & Test**

In [0]:
num_rows = 40
num_columns = 388
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

#  Membuat MODEL (**CNN MODEL**) 

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dense(num_labels, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# **Meng-compile Model**

In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# **Summary Model**

In [24]:
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 387, 16)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 193, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 193, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 192, 32)       2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 96, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 96, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 95, 64)         8

# **Training Model** ( dengan beberapa kali iterasi/ epoch )


model disimpan dengan nama 'sound_classification_Bagus.hdf5' *bisa disesuaikan sik


In [44]:
#tf.keras.callbacks.ModelCheckpoint

from datetime import datetime 

#num_epochs = 500
#num_batch_size = 128

#num_epochs = 500
#num_batch_size = 256

#num_epochs = 100
#num_batch_size = 256

num_epochs = 1000
num_batch_size = 256

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='sound_classification_Bagus.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)




Train on 99 samples, validate on 25 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 1.67510, saving model to sound_classification_Bagus.hdf5
99/99 [==============================] - 2s 16ms/sample - loss: 0.0273 - acc: 0.9798 - val_loss: 1.6751 - val_acc: 0.7200
Epoch 2/1000

Epoch 00002: val_loss did not improve from 1.67510
99/99 [==============================] - 1s 11ms/sample - loss: 0.0236 - acc: 0.9899 - val_loss: 1.7151 - val_acc: 0.7200
Epoch 3/1000

Epoch 00003: val_loss did not improve from 1.67510
99/99 [==============================] - 1s 11ms/sample - loss: 0.0123 - acc: 1.0000 - val_loss: 1.7262 - val_acc: 0.7200
Epoch 4/1000

Epoch 00004: val_loss did not improve from 1.67510
99/99 [==============================] - 1s 11ms/sample - loss: 0.0203 - acc: 0.9899 - val_loss: 1.6921 - val_acc: 0.7200
Epoch 5/1000

Epoch 00005: val_loss improved from 1.67510 to 1.64638, saving model to sound_classification_Bagus.hdf5
99/99 [==============================] - 

# **Melihat Akurasi**
dalam training pada case ini dilakukan beberapa kali hingga mendapatkan akurasi mendekati 0.9


In [46]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.97979796
Testing Accuracy:  0.8


# **SAVE MODEL**

In [0]:
model.save('my_model_BAGUS_SOUND.h5')

In [51]:
ls -lt | head - 10

==> standard input <==
total 126112
-rw------- 1 root root  585176 Dec 27 07:29 my_model_BAGUS_SOUND.h5
-rw------- 1 root root  585176 Dec 27 07:26 sound_classification_Bagus.hdf5
-rw------- 1 root root   77625 Dec 27 06:29 NEW_DATAX.csv
-rw------- 1 root root     520 Dec 26 10:02 list.txt
-rw------- 1 root root  793878 Dec 26 09:40 Copy of normal__201106221450.wav
-rw------- 1 root root  699994 Dec 26 09:40 Copy of normal__201108011118.wav
drwx------ 2 root root    4096 Dec 26 09:39 BUAT_SAMPLING_IMAGE/
-rw------- 1 root root  319205 Dec 26 02:42 DATAX.csv
-rw------- 1 root root  699994 Dec 23 06:54 normal__201108011118.wav
head: cannot open '10' for reading: No such file or directory


In [0]:

!cp my_model_BAGUS_SOUND.h5 LOAD_MODEL_27122019/my_model_BAGUS_SOUND.h5
!cp sound_classification_Bagus.hdf5 LOAD_MODEL_27122019/sound_classification_Bagus.hdf5

# **Membuat Fungsi Untuk Prediksi**

In [0]:
def buat_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [53]:
#/content/drive/My Drive/SOUND/set_b/Bunlabelledtest__101_1305030823364_A.wav

filename = '/content/drive/My Drive/SOUND/set_a/Aunlabelledtest__201012172010.wav' 
buat_prediction(filename)

The predicted class is: artifact 

artifact 		 :  1.00000000000000000000000000000000
extrahls 		 :  0.00000000000000424983577043441980
murmur 		 :  0.00000000000000000002360291215467
normal 		 :  0.00000000000000000000001443260785


In [54]:
filename = '/content/drive/My Drive/SOUND/set_a/Aunlabelledtest__201108222257.wav' 
buat_prediction(filename)

The predicted class is: murmur 

artifact 		 :  0.00000000116258580540318234852748
extrahls 		 :  0.00000000000000000000000091528868
murmur 		 :  0.99999737739562988281250000000000
normal 		 :  0.00000268036615125311072915792465


# **LOAD MODEL & FUNGSI PREDIKSI DENGAN LOAD MODEL** 

dari pada capek capek Training lagi running lagi ,mendingan kita load model yang udah kita save sebelumnya aja 

In [45]:
!pwd

/content/drive/My Drive/SOUND/set_a


In [0]:
new_model = tf.keras.models.load_model('my_model_BAGUS_SOUND.h5')

def LOAD_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = new_model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = new_model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

# **PREDIKSI**

In [57]:
filename = '/content/drive/My Drive/SOUND/set_a/Aunlabelledtest__201108222257.wav' 
LOAD_prediction(filename)

The predicted class is: murmur 

artifact 		 :  0.00000000116258580540318234852748
extrahls 		 :  0.00000000000000000000000091528868
murmur 		 :  0.99999737739562988281250000000000
normal 		 :  0.00000268036615125311072915792465
